# MIS-CPIMS Data Import Scripts

Import pandas library and read exported CPARA data from MIS (XLSX). Display all columns for exploration.

In [85]:
import pandas as pd
cpara=pd.read_excel("Data/CPARA.xlsx", sheet_name="Sheet1")
cpara_ovc_questions = pd.read_excel("Data/OVC_Questions.xlsx", sheet_name="Sheet1")
cpims_hhs=pd.read_excel("Data/Household ID OVC Mapping.xlsx", sheet_name="Sheet1")
cpara.columns

Index(['UnIque ID', 'record_id', 'assessment_date', 'cpims_id', 'case_manager',
       'child_headed', 'has_hei', 'has_pbf', 'has_svac', 'q1_1', 'q1_2',
       'q1_3', 'q1_4', 'benchmark1', 'q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5',
       'q2_6', 'q2_8', 'q2_7', 'q2_9', 'benchmark2', 'q3_1', 'q3_2', 'q3_3',
       'benchmark3', 'q4_1', 'q4_2', 'q4_3', 'q4_4', 'benchmark4', 'q5_1',
       'q5_2', 'q5_3', 'benchmark5', 'q6_1', 'q6_2', 'q6_3', 'q6_4', 'q6_5',
       'benchmark6', 'q7_1', 'q7_2', 'benchmark7', 'q8_1', 'benchmark8',
       'q9_1', 'q9_2', 'q9_3', 'q9_4', 'benchmark9', 'healthy', 'stable',
       'safe', 'schooled', 'entry_time', 'action_by', 'designation', 'lip',
       'entry_by_name', 'client_time', 'deleted', 'modified', 'gps', 'device',
       'cpimsId', 'names', 'caregiverhivstatus', 'cbo_id', 'cbo', 'county',
       'subcounty', 'ward', 'chv_id', 'chv'],
      dtype='object')

In [86]:
import uuid

print(uuid.uuid4())

record_counts = cpara['record_id'].count()



def generate_uid(count):
    uid_list = []
    for i in range (0, count):
        uuid_gen = uuid.uuid4()
        print(f'generated uid: {uuid_gen} inte: {i}')
        uid_list.append(uuid_gen)
    return uid_list
    

df_uid = pd.DataFrame(generate_uid(record_counts))

bbb10137-08a5-4039-a599-6fa908d5aa9a
generated uid: 319fbb67-7e63-4253-8e57-f90009219ea3 inte: 0
generated uid: de05d872-8686-4b89-982d-e385c782831a inte: 1
generated uid: 74ddc293-f0cf-4219-a075-36a6f16592d7 inte: 2
generated uid: e9dde29a-1583-4964-adc5-fe4923ee977a inte: 3
generated uid: 62525da8-db02-48a0-b57e-b2126cd4b49b inte: 4
generated uid: 2075af48-5d52-4bf2-bb44-439a0c326555 inte: 5
generated uid: 40223de5-40a6-447f-ba8a-67b11b61a121 inte: 6
generated uid: f1a75fc2-ed63-4e45-a311-df5b93724427 inte: 7
generated uid: 6a506ac6-d7e0-4dbf-8aa6-4e7ef936dfb6 inte: 8
generated uid: e0a92a6c-c02d-495d-86b1-e32b0af6ffd7 inte: 9
generated uid: 11a14950-b69f-47a2-be3e-b38236d0674f inte: 10
generated uid: 0bbda7fa-4a27-484b-b13e-ccb717b92b3b inte: 11
generated uid: fee69d4e-8730-405c-a689-b38d67bf4be4 inte: 12
generated uid: 3543e455-d6c0-4ac5-9180-4d223c972cdd inte: 13
generated uid: b0cad76d-911e-4cc5-ac01-2d164973fdc6 inte: 14
generated uid: 84df9792-422e-4664-be40-e18832112dba inte: 

In [87]:
print(df_uid)

                                          0
0      319fbb67-7e63-4253-8e57-f90009219ea3
1      de05d872-8686-4b89-982d-e385c782831a
2      74ddc293-f0cf-4219-a075-36a6f16592d7
3      e9dde29a-1583-4964-adc5-fe4923ee977a
4      62525da8-db02-48a0-b57e-b2126cd4b49b
...                                     ...
32555  1d6e0df3-4f35-47c4-a1c3-c5f0b3aa42f6
32556  531e46b2-0006-40fb-909a-39b04dd639d2
32557  f4507d38-1292-4a55-9559-824f944e43d0
32558  8ffbba31-2477-4e10-bb6b-ad5d1fc83d82
32559  8ab90281-1306-426d-8fe9-26cb8a514983

[32560 rows x 1 columns]


In [88]:
cpims_questions=pd.read_excel("Data/ovc_care_questions.xlsx", sheet_name="Sheet1")

In [89]:
cpara_data=pd.merge(cpara, cpims_hhs, how='left', left_on='cpims_id', right_on='caregiver_id')
cpara_data1 = pd.merge(cpara_data, df_uid, left_index=True, right_index=True)
cpara_data1.head(2)

,UnIque ID,record_id,assessment_date,cpims_id,case_manager,child_headed,has_hei,has_pbf,has_svac,q1_1,...,cbo,county,subcounty,ward,chv_id,chv,cpims_ovc_id,caregiver_id,household,0
0,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282788.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,319fbb67-7e63-4253-8e57-f90009219ea3
1,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282807.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,de05d872-8686-4b89-982d-e385c782831a


In [90]:
cpara_ovc_questions.columns

Index(['record_id', 'cpara_record_id', 'cpara_date', 'question', 'response',
       'cpims_id', 'caregiver_id', 'chv_id'],
      dtype='object')

In [91]:
cpara_individual=pd.merge(cpara_ovc_questions, cpims_hhs, how='left', left_on='caregiver_id', right_on='caregiver_id')
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [92]:
cpara_ovc_questions.dtypes #cpims_hhs.dtypes

record_id           int64
cpara_record_id     int64
cpara_date         object
question           object
response           object
cpims_id            int64
caregiver_id        int64
chv_id              int64
dtype: object

Align the MIS data from rows to columns => vertical and only display the first two records. Move the columns that should not be considered as var.

In [93]:
cpara_unpivot=pd.melt(cpara_data, id_vars=['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name','client_time', 'modified', 'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household'])

cpara_unpivot.head(2)

,record_id,assessment_date,cpims_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,deleted,gps,device,case_manager,cpims_ovc_id,household,variable,value
0,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282788.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,UnIque ID,2022-09-12-105151
1,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282807.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,UnIque ID,2022-09-12-105151


In [94]:
cpara_unpivot.columns

Index(['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name', 'client_time', 'modified',
       'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household',
       'variable', 'value'],
      dtype='object')

In [95]:
ovc_care_benchmark = cpara_unpivot[cpara_unpivot["variable"].str.contains('benchmark')]#.pivot(index=['record_id', 'cpims_id','assessment_date','entry_time', 'modified', 'cpims_ovc_id', 'household'], columns='variable', values=['value'])
# ovc_care_benchmark.to_excel("ovc_care_benchmark.xlsx")
print(type(ovc_care_benchmark))
ovc_care_benchmark.head(2)

<class 'pandas.core.frame.DataFrame'>


,record_id,assessment_date,cpims_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,deleted,gps,device,case_manager,cpims_ovc_id,household,variable,value
832923,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282788.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,benchmark1,Yes
832924,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282807.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,benchmark1,Yes


In [96]:
# convert yes to 1 , no to 0, [["benchmark1","benchmark2","benchmark3","benchmark4","benchmark5","benchmark6","benchmark7","benchmark8","benchmark9",]] 

ovc_care_benchmark = ovc_care_benchmark.replace({"Yes": 1, "No": 0})

ovc_care_benchmark.head(2)

# ovc_care_benchmark.columns




 


,record_id,assessment_date,cpims_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,deleted,gps,device,case_manager,cpims_ovc_id,household,variable,value
832923,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282788.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,benchmark1,1
832924,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282807.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,benchmark1,1


In [97]:
# Insert missing colums into ovc_benchmark from schema

import numpy as np
import pandas as pd


# read df
# ovc_care_benchmark=pd.read_excel("ovc_care_benchmark.xlsx")


# Rename columns in benchmark to column names in benchmark schema
# Rename the 'household' column to 'household_id'
ovc_care_benchmark = ovc_care_benchmark.rename(columns={'household': 'household_id'})

# Rename the 'cpims_id' column to 'care_giver_id'
ovc_care_benchmark = ovc_care_benchmark.rename(columns={'cpims_id': 'care_giver_id'})

# Rename the 'assessment_date' column to 'date_of_event'
ovc_care_benchmark = ovc_care_benchmark.rename(columns={'assessment_date': 'date_of_event'})


ovc_care_benchmark[["benchmark10","benchmark11","benchmark12","benchmark13","benchmark14","benchmark15","benchmark16","benchmark17"]]=np.nan



ovc_care_benchmark[["bench_mark_score_id"] ]= np.nan

ovc_care_benchmark[["timestamp_updated"]]= None

ovc_care_benchmark[["timestamp_created"]]= None


ovc_care_benchmark.head(2)









,record_id,date_of_event,care_giver_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,...,benchmark11,benchmark12,benchmark13,benchmark14,benchmark15,benchmark16,benchmark17,bench_mark_score_id,timestamp_updated,timestamp_created
832923,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
832924,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [98]:
# pivot the table ovc_care_benchmark

ovc_care_benchmark = ovc_care_benchmark[ovc_care_benchmark["variable"].str.contains('benchmark')].pivot(index=['record_id', 'care_giver_id','date_of_event','entry_time', 'modified', 'cpims_ovc_id', 'household_id',"benchmark11","benchmark12","benchmark13","benchmark14","benchmark15","benchmark16","benchmark17","bench_mark_score_id","timestamp_updated","timestamp_created"], columns='variable', values=['value'])

# num_rows = ovc_care_benchmark['record_id'].count()

ovc_care_benchmark = ovc_care_benchmark.reset_index().drop(['variable', 'value'], axis=1)
ovc_care_benchmark = ovc_care_benchmark.melt(id_vars=['record_id', 'care_giver_id', 'date_of_event', 'entry_time', 'modified', 'cpims_ovc_id', 'household_id', 'benchmark11', 'benchmark12', 'benchmark13', 'benchmark14', 'benchmark15', 'benchmark16', 'benchmark17', 'bench_mark_score_id', 'timestamp_updated', 'timestamp_created'], var_name='benchmark', value_name='value')

# print(num_rows)

ovc_care_benchmark.head(2)

,record_id,care_giver_id,date_of_event,entry_time,modified,cpims_ovc_id,household_id,benchmark11,benchmark12,benchmark13,benchmark14,benchmark15,benchmark16,benchmark17,bench_mark_score_id,timestamp_updated,timestamp_created,benchmark,value
0,9325,105151,2022-09-12,2022-09-27 12:49:56,2022-09-27 12:49:56,4282788.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,index,832923
1,9325,105151,2022-09-12,2022-09-27 12:49:56,2022-09-27 12:49:56,4282807.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,index,832924


In [105]:
# insert auto generated uuids into bench_mark_score_id

record_counts = ovc_care_benchmark['record_id'].count()


print(record_counts)
"""def generate_uid(count):
    uid_list = []
    for i in range (0, count):
        uuid_gen = uuid.uuid4()
        print(f'generated uid: {uuid_gen} inte: {i}')
        uid_list.append(uuid_gen)
    return uid_list
    

df_uid = pd.DataFrame(generate_uid(record_counts)) """

# df_uid = pd.DataFrame(generate_uid(record_counts)) 



9162153


"def generate_uid(count):\n    uid_list = []\n    for i in range (0, count):\n        uuid_gen = uuid.uuid4()\n        print(f'generated uid: {uuid_gen} inte: {i}')\n        uid_list.append(uuid_gen)\n    return uid_list\n    \n\ndf_uid = pd.DataFrame(generate_uid(record_counts)) "

In [100]:
#ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')]
ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')].pivot(index=['record_id', 'cpims_id','assessment_date','cpims_ovc_id', 'household'], columns='variable', values=['value'])
ovc_care_cpara.to_csv("ovc_care_cpara_pivot.csv")
ovc_care_cpara.head(2)

value            \
variable                                                   q1_1 q1_2 q1_3   
record_id cpims_id assessment_date cpims_ovc_id household                   
1         3273808  2022-09-07      NaN          NaN          na  Yes   na   
51        3075340  2022-09-06      NaN          NaN          na  Yes   na   

                                                                               \
variable                                                  q1_4 q2_1 q2_2 q2_3   
record_id cpims_id assessment_date cpims_ovc_id household                       
1         3273808  2022-09-07      NaN          NaN        Yes   na   na   na   
51        3075340  2022-09-06      NaN          NaN        Yes   na   na   na   

                                                                          ...  \
variable                                                  q2_4 q2_5 q2_6  ...   
record_id cpims_id assessment_date cpims_ovc_id household                 ...   
1         3273808  2022-09-07      NaN          NaN         na   na   na  ...   
51        3075340  2022-09-06      NaN          NaN         na   na   na  ...   

                                                                               \
variable                                                  q6_3 q6_4 q6_5 q7_1   
record_id cpims_id assessment_date cpims_ovc_id household                       
1         3273808  2022-09-07      NaN          NaN         No   na   na  Yes   
51        3075340  2022-09-06      NaN          NaN         No   na   na  Yes   

                                                                               \
variable                                                  q7_2 q8_1 q9_1 q9_2   
record_id cpims_id assessment_date cpims_ovc_id household                       
1         3273808  2022-09-07      NaN          NaN        Yes  Yes  Yes  Yes   
51        3075340  2022-09-06      NaN          NaN        Yes  Yes  Yes   No   

                                                                     
variable                                                  q9_3 q9_4  
record_id cpims_id assessment_date cpims_ovc_id household            
1         3273808  2022-09-07      NaN          NaN         na  Yes  
51        3075340  2022-09-06      NaN          NaN         na  Yes  

[2 rows x 35 columns]

In [101]:
cpara_individual.to_excel("cpara_ovc_questions.xlsx")
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [102]:
cpims_hhs.columns

Index(['cpims_ovc_id', 'caregiver_id', 'household'], dtype='object')

In [103]:
cpara_data.to_excel("CPARA_MAPPING.xlsx")

In [104]:
cpara_data.head(2)

,UnIque ID,record_id,assessment_date,cpims_id,case_manager,child_headed,has_hei,has_pbf,has_svac,q1_1,...,cbo_id,cbo,county,subcounty,ward,chv_id,chv,cpims_ovc_id,caregiver_id,household
0,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,2796.0,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282788.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a
1,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,2796.0,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282807.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a
